In [9]:
import gmaps
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from api_keys import g_key
import requests
#from google.colab import output
#output.enable_custom_widget_manager()

In [10]:
weather_df = pd.read_csv("weather.csv")
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.193952,168.865478,55.80,55,100,2.55,NZ,1662940196
1,1,zhigansk,66.752064,123.394112,36.86,83,41,9.69,RU,1662940197
2,2,butaritari,3.116667,172.800000,82.06,71,89,10.74,KI,1662940197
3,3,punta arenas,-53.163384,-70.907826,43.09,61,40,17.27,CL,1662940198
4,4,rikitea,-23.122303,-134.969154,73.13,73,3,5.41,PF,1662940199


In [11]:
locations = weather_df[["Lat" , "Lng"]]
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(locations , weights=weather_df['Humidity'],max_intensity=100,point_radius=30)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
vacation_df = weather_df.loc[(weather_df["Max Temp"]>70 )&(weather_df["Max Temp"]<80)&(weather_df["Wind Speed"]<10)&(weather_df["Cloudiness"] == 0)]
vacation_df = vacation_df.reset_index()
hotel_df= pd.DataFrame()
hotel_df["City"]= vacation_df["City"]
hotel_df["Lat"] = vacation_df["Lat"]
hotel_df["Lng"] = vacation_df["Lng"]
hotel_df["Country"] = vacation_df["Country"]
hotel_df["Hotel Name"] = ""
hotel_df["Rating"] = ""
vacation_df.head()

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,178,178,fortuna,40.598187,-124.157276,71.92,79,0,8.05,US,1662940293
1,190,190,nador,35.168616,-2.927584,77.25,88,0,3.44,MA,1662940299
2,206,206,shangqiu,34.414270,115.656350,73.38,64,0,2.26,CN,1662940307
3,241,241,los llanos de aridane,28.657722,-17.912202,71.26,84,0,5.99,ES,1662940325
4,293,293,cayenne,4.922420,-52.313453,76.91,88,0,0.00,GF,1662940353


In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "", 
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,}
for index, row in vacation_df.iterrows():
    params["location"] = f"{row['Lat']},{row['Lng']}"
    response = requests.get(base_url, params=params).json()
    result = response["results"][0]
    print(response["results"][0].keys())
    try:
        name = response["results"][0]["name"]
        print(f"Closest hotel is {name}.")
        
        hotel_df.loc[index, 'Hotel Name'] = result['name']
        hotel_df.loc[index, 'address'] = result['vicinity']
        hotel_df.loc[index, 'rating'] = result['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        print("------------")


dict_keys(['business_status', 'geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'name', 'place_id', 'plus_code', 'reference', 'scope', 'types', 'vicinity'])
Closest hotel is sam Glass trucking.
Missing field/result... skipping.
------------
dict_keys(['business_status', 'geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'name', 'opening_hours', 'place_id', 'plus_code', 'reference', 'scope', 'types', 'vicinity'])
Closest hotel is BioCoin Restaurant.
Missing field/result... skipping.
------------
dict_keys(['business_status', 'geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'name', 'place_id', 'reference', 'scope', 'types', 'vicinity'])
Closest hotel is 商丘市纪检委.
Missing field/result... skipping.
------------
dict_keys(['business_status', 'geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'name', 'opening_hours', 'photos', 'place_id', 'plus_code', 'rating', 'reference', 'scope', 'types', 'user_ratings_total', 'vicinity'])
C

In [14]:
hotel_df
# NOTE: Do not change any of the code in this cell 
#i changed code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))